# Just defining same as main_learning_ppo

In [ ]:
from printind.printind_function import printi, printiv
from Env_instance import resume_env, nb_actuations

import numpy as np
from tensorforce.agents import PPOAgent
from tensorforce.execution import Runner
import os
import csv

In [ ]:
printi("resume env")

environment = resume_env(plot=2000, step=100, dump=100, first_run=False)
deterministic=True

printi("define network specs")

network_spec = [
    dict(type='dense', size=512),
    dict(type='dense', size=512),
]

printi("define agent")

printiv(environment.states)
printiv(environment.actions)
printiv(network_spec)

In [ ]:
agent = PPOAgent(
    states=environment.states(),
    actions=environment.actions(),
    network=network_spec,
    # Agent
    states_preprocessing=None,
    actions_exploration=None,
    reward_preprocessing=None,
    # MemoryModel
    update_mode=dict(
        unit='episodes',
        # 10 episodes per update
        batch_size=20,
        # Every 10 episodes
        frequency=20
    ),
    memory=dict(
        type='latest',
        include_next_states=False,
        capacity=10000
    ),
    # DistributionModel
    distributions=None,
    entropy_regularization=0.01,
    # PGModel
    baseline_mode='states',
    baseline=dict(
        type='mlp',
        sizes=[32, 32]
    ),
    baseline_optimizer=dict(
        type='multi_step',
        optimizer=dict(
            type='adam',
            learning_rate=1e-3
        ),
        num_steps=5
    ),
    gae_lambda=0.97,
    # PGLRModel
    likelihood_ratio_clipping=0.2,
    # PPOAgent
    step_optimizer=dict(
        type='adam',
        learning_rate=1e-3
    ),
    subsampling_fraction=0.2,
    optimization_steps=25,
    execution=dict(
        type='single',
        session_config=None,
        distributed_spec=None
    )
)

In [ ]:
def episode_finished(r):
    print("Finished episode {ep} after {ts} timesteps (reward: {reward})".format(ep=r.episode, ts=r.episode_timestep,
                                                                                 reward=r.episode_rewards[-1]))

    printi("save the mode")

    name_save = "./saved_models/ppo_model"
    # NOTE: need to check if should create the dir
    r.agent.save_model(name_save, append_timestep=False)

    # show for plotting
    # r.environment.show_control()
    # r.environment.show_drag()

    # print(sess.run(tf.global_variables()))

    return True

# Restoring the best model that was learned

In [ ]:
agent.restore_model('best_model')

In [ ]:
runner = Runner(agent=agent, environment=environment)

In [ ]:
environment.show_flow()

In [ ]:
from dolfin import *
with XDMFFile(MPI.comm_world, 'u_.xdmf') as file:
    file.write(environment.u_, 0.)

In [ ]:
crrt_drag = environment.history_parameters["drag"].get()

In [ ]:
plt.figure()
plt.plot(crrt_drag, label="episode drag", linewidth=1.2)
plt.plot([0, environment.size_history - 1], [environment.inspection_params['line_drag'], environment.inspection_params['line_drag']], label="mean drag no control", linewidth=2.5, linestyle="--")
plt.ylabel("measured drag D")
plt.xlabel("actuation step")
range_drag_plot = environment.inspection_params["range_drag_plot"]
plt.legend(loc=2)
#plt.ylim(range_drag_plot)
plt.tight_layout()
#plt.pause(1.0)
plt.savefig('drag')